In [1]:
import mediapipe as mp
import cv2
# import mediapipe as mp
import numpy as np
import pandas as pd
import os
import csv
import time
import math


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles
# help(mp.solutions.drawing_utils)

In [4]:
# def image_resize(image):
# #     screen_res = 1760, 990
#     screen_res = 1280, 720
#     scale_width = screen_res[0] / image.shape[1]
#     scale_height = screen_res[1] / image.shape[0]
#     scale = min(scale_width, scale_height)
#     #resized window width and height
#     window_width = int(image.shape[1] * scale)
#     window_height = int(image.shape[0] * scale)
#     resized_img=cv2.resize(image, (window_width,window_height))
#     return resized_img

def find_mid(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return (a+b)/2

def distance(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return ( ((a[0]-b[0])**2) + ((a[1]-b[1])**2) + ((a[2]-b[2])**2) ) **0.5

def magnitude(a):
    a = np.array(a)
    return ((a[0]**2)+(a[1]**2)+(a[2]**2))**0.5

def get_line(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return a-b
         
def get_joints(landmarks):
#     joints = head(mid point of eyes), neck, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, mid hip, right hip, left knee, right knee, left ankle, right ankle
    joints=[]
    # print("landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x")
    # print(landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x)
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].z],
                            [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].z])
                  )
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]))
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]))
    
    return joints

def get_body_lines(joints):
    body_lines=[]
    body_lines.append(get_line(joints[0],joints[1])) # head to neck
    body_lines.append(get_line(joints[1],joints[9])) # neck to hip
    body_lines.append(get_line(joints[8],joints[9])) # left_hip to mid_hip
    body_lines.append(get_line(joints[10],joints[9])) # right_hip to mid_hip
    body_lines.append(get_line(joints[8],joints[11])) # left_hip to left_knee
    body_lines.append(get_line(joints[10],joints[12])) # right_hip to right_knee
    body_lines.append(get_line(joints[11],joints[13])) # left_knee to left_ankle
    body_lines.append(get_line(joints[12],joints[14])) # right_knee to right_ankle
    body_lines.append(get_line(joints[2],joints[1])) # left_shoulder to neck
    body_lines.append(get_line(joints[3],joints[1])) # right_shoulder to neck
    body_lines.append(get_line(joints[2],joints[4])) # left_shoulder to left_elbow
    body_lines.append(get_line(joints[4],joints[6])) # left_elbow to left_wrist
    body_lines.append(get_line(joints[3],joints[5])) # right_shoulder to right_elbow
    body_lines.append(get_line(joints[5],joints[7])) # right_elbow to right_wrist
#     for i in range(0,len(joints)):
#         for j in range(i+1,len(joints)):
#             body_lines.append(get_line(joint[i],joints[j]))
    # print("body_lines")
    # print(len(body_lines))
    # print(body_lines)
    return body_lines

def get_body_angles(body_lines):
    body_angles=[]
    for i in range(0,len(body_lines)):
        for j in range(i+1,len(body_lines)):
            # print("i,j ")
            # print(i)
            # print(j)
            numerator=np.dot(body_lines[i],body_lines[j])
            # print("numerator")
            # print(numerator)
            denominator=magnitude(body_lines[i])*magnitude(body_lines[j])
            # print("denominator")
            # print(denominator)
            argument = max(-1.0, min(numerator / denominator, 1.0))
            body_angles.append(math.acos(argument))
            # print("No eerrrorr")
    return body_angles  

def calculate_features(image):
    output=[]
    # Checking if the image is empty or not
    if image is None:
        result = "Image is empty!!"
        return
    # image=image_resize(image)
    
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(image)
        # print("results")
        # print(results)
        if not results.pose_landmarks:
            return
        try:
            landmarks = results.pose_landmarks.landmark
            # print("landmarks")
            # print(landmarks)
            output=[]
            joints=get_joints(landmarks)
            # print("joints")
            # print(joints)
#             output=output+joints
            # get_body_lines(joints)
            bodyLines=get_body_lines(joints)
            output=output+get_body_angles(bodyLines)
            # print("output")
            # print(output)
            return output
        except Exception as e:
            print("Error in calculateFetaure :")
            print(e)
            

In [5]:
dataset=[]
count=0
errors=0


# image_path = 'Trikon.png'
# img = cv2.imread(image_path)
# image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# entry=calculate_features(image)
# print("Entry")

video_folder = 'Test Videos/Bhujangasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    # i=1
    # while(i>0):
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        # i = i-1
        if not success:
            break
        # print("Inamgs")
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            # # Convert the image to RGB (MediaPipe requires RGB images)
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            # print("Entry")
            # print(entry)
            if(entry is not None):
                entry.append("Bhujangasana")
                dataset.append(entry)
            else:
                errors=errors+1
        # print("dataset")
        # print(dataset)
    cap.release()
        

video
video
video


In [6]:
len(dataset)

679

In [7]:
video_folder = 'Test Videos/Janu Sirasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("JanuSirasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video


In [8]:
len(dataset)

1410

In [9]:
video_folder = 'Test Videos/Padmasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Padmasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video
video
video
video


In [10]:
len(dataset)

6775

In [11]:
video_folder = 'Test Videos/savasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Savasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video


In [12]:
len(dataset)

9318

In [13]:
video_folder = 'Test Videos/Tadasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Tadasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video


In [14]:
len(dataset)

10342

In [15]:
video_folder = 'Test Videos/Trikonasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Trikonasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video


In [16]:
len(dataset)

13711

In [17]:
dataset[2]
len(dataset[2])

92

In [18]:
import pandas as pd
columns=[]
for i in range(1,92):
    columns.append('feature_'+str(i))

columns.append('Asana')
print(len(columns))
print(columns)
df = pd.DataFrame(dataset, columns = columns)
df

92
['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71', 'feature_72

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,Asana
0,0.223431,1.771471,1.370122,1.435084,1.230607,0.770015,1.516022,1.772057,1.369536,0.413056,...,1.939147,1.924743,1.631128,0.606364,0.575178,0.316614,0.118093,0.308019,0.318283,Bhujangasana
1,0.209558,1.764170,1.377423,1.351904,1.203542,0.660020,1.264804,1.787550,1.354043,0.435408,...,2.003749,1.935247,1.686884,0.614988,0.534662,0.306813,0.123962,0.317095,0.266401,Bhujangasana
2,0.188879,1.617645,1.523947,0.903256,1.121439,1.332067,1.324038,1.640867,1.500726,0.416719,...,2.365349,1.900060,1.640250,0.845871,0.361269,0.152990,0.489782,0.729419,0.275517,Bhujangasana
3,0.155691,1.651358,1.490235,0.941001,1.090989,1.278930,1.310436,1.677749,1.463843,0.438362,...,2.379770,1.890193,1.653081,0.841437,0.336064,0.133645,0.508987,0.731519,0.247292,Bhujangasana
4,0.160462,1.678798,1.462795,0.943511,1.099220,1.257900,1.319687,1.694980,1.446612,0.441860,...,2.335868,1.884487,1.650557,0.793069,0.324264,0.125797,0.473472,0.690895,0.243838,Bhujangasana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13706,0.245818,2.525476,0.616116,1.942578,1.626298,1.238048,0.786594,2.125960,1.015633,1.657962,...,1.501678,3.052409,2.963807,0.817072,2.513018,2.260243,1.724702,1.464611,0.260097,Trikonasana
13707,0.241981,2.537400,0.604193,1.862499,1.562497,1.178199,0.756945,2.104897,1.036695,1.710477,...,1.484549,3.053263,2.968939,0.769309,2.483725,2.234721,1.742137,1.486443,0.255709,Trikonasana
13708,0.244925,2.516208,0.625385,1.891176,1.533322,1.180666,0.741684,2.074341,1.067252,1.728404,...,1.489090,3.072070,2.914429,0.789866,2.478989,2.195587,1.720489,1.426890,0.293839,Trikonasana
13709,0.256506,2.578924,0.562669,1.955195,1.666437,1.202603,0.814067,2.187576,0.954017,1.758134,...,1.643733,3.107671,2.919952,0.795614,2.293028,2.057166,1.521143,1.278958,0.242413,Trikonasana


In [19]:
df.to_csv('TestData.csv',index=False)
df.shape

(13711, 92)